In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [11]:
data=pd.read_csv(r'D:\Task\IMDB Dataset.csv')

In [12]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [13]:
data.replace ({"sentiment":{"positive":1,"negative":0}}, inplace=True)
data.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_11744\1367238801.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace ({"sentiment":{"positive":1,"negative":0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [14]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape, test_data.shape)

(40000, 2) (10000, 2)


In [15]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(train_data["review"])

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [16]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [17]:
embedding_dim = 32
vocab_size = 5000

model = Sequential([
    Embedding(vocab_size, embedding_dim, name="embedding"),
    Bidirectional(LSTM(64)),
    Dropout(0.2), 
    Dense(1, activation='sigmoid')
])

In [18]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [19]:
history = model.fit(x_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

Epoch 1/3
1000/1000 [==============================] - 256s 247ms/step - loss: 0.3970 - accuracy: 0.8209 - val_loss: 0.2898 - val_accuracy: 0.8783
Epoch 2/3
1000/1000 [==============================] - 249s 249ms/step - loss: 0.2812 - accuracy: 0.8886 - val_loss: 0.3564 - val_accuracy: 0.8702
Epoch 3/3
1000/1000 [==============================] - 246s 246ms/step - loss: 0.2342 - accuracy: 0.9089 - val_loss: 0.3071 - val_accuracy: 0.8767


In [20]:
def predict_sentiment(review):

  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)


  prediction = model.predict(padded_sequence)
  print(prediction)


  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [21]:
while True:
  new_review = input("Enter a movie review (or type 'exit' to quit): ")
  if new_review.lower() == "exit":
    break
  sentiment = predict_sentiment(new_review)
  print("Sentiment of the review is:", sentiment)

Enter a movie review (or type 'exit' to quit):  exit
